# **Business Optimisation Analysis of Chinook Digital Music Store**

Business optimisation is the process of improving the efficiency, productivity and performance of an organisation. It is the idea of consistently striving for better outcomes. This can apply both to internal operations and external products. Key elements of business optimisation include:

- Measurement of productivity, efficiency and performance
- Identifying areas for improvement
- Introducing new methods and processes
- Measuring and comparing results
- Repeating the cycle

Some examples of business optimisation include:

- Introducing new methods, practices and systems that reduce turnaround time
- Reducing costs while improving performance
- Automation of repetitive tasks
- Increasing sales through enhancing customer satisfaction etc.

Hence, in this project I have conducted an analysis for Chinook, an imaginary digital music store, to help optimise their business. The database has been collected from the [github repo](https://github.com/lerocha/chinook-database) of [Luis Rocha](https://github.com/lerocha).

The analysis done can help the company in identifying:

- Most profitable genres
- Performance of sales support agents
- Suitable marketing campaign to boost sales in least profitable countries
- Best city to organise a music festival in
- Effect of changing purchasing strategy to save money
- Sales boosting strategy through highest selling artists
- Type of concert to hold and the artists to invite that would maximise participation
- Customer reward system for the top spenders in each country

## \*\*Use Chinook Database\*\*

In [1]:
USE Chinook;
GO

Commands completed successfully.

Total execution time: 00:00:00.034

## Most Profitable Genres

The Chinook record store has just signed a deal with a new record label, and we need to select the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

- Regal _Hip-Hop_
- Red Tone _Punk_
- Meteor and the Girls _Pop_
- Slim Jim Bites _Blues_

Let's find the top genres in Chinook.

In [2]:
SELECT Genre.Name AS genre, SUM(InvoiceLine.Quantity) AS tracks_sold, SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS total_sales_USD
FROM InvoiceLine
JOIN Track 
ON InvoiceLine.TrackId = Track.TrackId 
JOIN Genre 
ON Track.GenreId = Genre.GenreId
GROUP BY Genre.Name
ORDER BY tracks_sold DESC;

(24 rows affected)

Total execution time: 00:00:00.202

genre,tracks_sold,total_sales_USD
Rock,835,826.65
Latin,386,382.14
Metal,264,261.36
Alternative & Punk,244,241.56
Jazz,80,79.20
Blues,61,60.39
TV Shows,47,93.53
R&B/Soul,41,40.59
Classical,41,40.59
Reggae,30,29.70


Based on the sales of tracks across different genres, we should definitely add the album by Red Tone (Punk), and may also add the albums by Slim Jim Bites (Blues) and Meteor and the Girls (Pop).

## Performance of Sales Support Agents

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We need to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [3]:
WITH total_sales_rep AS 
(
    SELECT c.SupportRepId, SUM(i.Total) AS total_sales_USD
    FROM Customer c
    LEFT JOIN Invoice i
    ON c.CustomerId = i.CustomerId 
    GROUP BY c.SupportRepId 
)

SELECT (e.FirstName + ' ' +  e.LastName) AS Name, e.Title, e.HireDate, ts.total_sales_USD
FROM Employee e
JOIN total_sales_rep ts
ON e.EmployeeId = ts.SupportRepId 
ORDER BY ts.total_sales_USD DESC;

(3 rows affected)

Total execution time: 00:00:00.035

Name,Title,HireDate,total_sales_USD
Jane Peacock,Sales Support Agent,2002-04-01 00:00:00.000,833.04
Margaret Park,Sales Support Agent,2003-05-03 00:00:00.000,775.40
Steve Johnson,Sales Support Agent,2003-10-17 00:00:00.000,720.16


While there is about 14% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## Country Sales Analytics

We wish to find the least profitable countries, i.e. countries that generated the least revenue, so that we can run a marketing campaign to boost sales there.

In [4]:
SELECT BillingCountry AS Country, COUNT(*) AS total_invoices, SUM(Total) AS total_sales_USD
FROM Invoice
GROUP BY BillingCountry
ORDER BY total_sales_USD DESC;

(24 rows affected)

Total execution time: 00:00:00.007

Country,total_invoices,total_sales_USD
USA,91,523.06
Canada,56,303.96
France,35,195.10
Brazil,35,190.10
Germany,28,156.48
United Kingdom,21,112.86
Czech Republic,14,90.24
Portugal,14,77.24
India,13,75.26
Chile,7,46.62


From the table, we can find a lot of ties for the country with least revenue. The sales manager wishes to target any country generating less than $40 for the marketing campaign.

## Music Festival

The investors of Chinook would like to throw a promotional Music Festival in the city they made the most money. Let's find the city with the highest sales.

In [5]:
SELECT TOP(1) WITH TIES BillingCity AS City, BillingCountry AS Country, SUM(Total) AS total_sales_USD
FROM Invoice
GROUP BY BillingCity, BillingCountry
ORDER BY total_sales_USD DESC;

(1 row affected)

Total execution time: 00:00:00.023

City,Country,total_sales_USD
Prague,Czech Republic,90.24


Congratulations Prague! You guys got yourself a night to remember as appreciation for your support for Chinook.

## Changing Purchase Strategy

The Chinook store allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [6]:
-- no. of tracks bought per album per invoice
WITH invoice_track_album AS 
(
    SELECT il.InvoiceId, t.AlbumId, COUNT(*) AS tracks_bought
    FROM InvoiceLine il 
    JOIN Track t 
    ON il.TrackId = t.TrackId
    GROUP BY il.InvoiceId, t.AlbumId
),

-- no. of tracks per album
album_tracks AS
(
    SELECT AlbumId, COUNT(*) AS total_tracks
    FROM Track
    GROUP By AlbumId
),

-- full or partial album per invoice
partial_full AS 
(
    SELECT ita.InvoiceId, ita.AlbumId,
    CASE WHEN (
        SELECT(at.total_tracks - ita.tracks_bought)
    ) = 0 THEN 'full'
    ELSE 'partial'
    END AS category
    FROM invoice_track_album ita
    JOIN album_tracks at
    ON ita.AlbumId = at.AlbumId
)

SELECT 
    category, 
    COUNT(*) AS number_sold, 
    ROUND(1.0 * COUNT(*) / (SELECT COUNT(*) FROM partial_full) * 100, 2) AS percent_sold 
    FROM partial_full
    GROUP BY category;

(2 rows affected)

Total execution time: 00:00:00.016

category,number_sold,percent_sold
partial,1254,96.240000000000
full,49,3.760000000000


Album purchases account for only 3.76% of total purchases. <span style="color: rgba(0, 0, 0, 0.7); font-family: Inter, sans-serif; background-color: rgb(255, 255, 255);">Based on this data, it's possible to recommend purchasing&nbsp;</span> <span style="color: rgba(0, 0, 0, 0.7); font-family: Inter, sans-serif; background-color: rgb(255, 255, 255);">only select tracks from albums from record companies as there will be less to lose.</span>

## Popular Playlist Artists

To boost growth, the management of Chinook wishes to add tracks from albums of artists who are most popular in customers' playlists, so that there are more purchases of popular artists. Let's help them by finding the top 5 artists who are more popular among the customers.

In [7]:
SELECT TOP(5) ar.Name, COUNT(*) AS time_in_playlists
FROM PlaylistTrack pt 
JOIN Track t 
ON pt.TrackId = t.TrackId
JOIN Album al 
ON t.AlbumId = al.AlbumId
JOIN Artist ar 
ON ar.ArtistId = al.ArtistId
GROUP BY ar.Name
ORDER BY time_in_playlists DESC;

(5 rows affected)

Total execution time: 00:00:00.014

Name,time_in_playlists
Iron Maiden,516
U2,333
Metallica,296
Led Zeppelin,252
Deep Purple,226


We have our top artists. Time to approach them for more songs to boost our sales.

## Countrywise Popular Genre

The management of Chinook wishes to find the most popular song genre in each country in order to do some further analytics regarding which genre to invest more on marketing in each country. We determine the most popular genre as the genre with the highest number of purchases.

In [8]:
WITH pop_genre_country AS 
(
    SELECT 
        c.Country, 
        g.Name, 
        COUNT(*) AS total_sold, 
        RANK() OVER(PARTITION BY c.Country ORDER BY COUNT(*) DESC) AS rnk
        FROM CUSTOMER c 
        JOIN Invoice i 
        ON c.CustomerId = i.CustomerId
        JOIN InvoiceLine il 
        ON i.InvoiceId = il.InvoiceId
        JOIN Track t 
        ON il.TrackId = t.TrackId
        JOIN Genre g 
        ON t.GenreId = g.GenreId

    GROUP BY c.Country, g.GenreId, g.Name
)

SELECT Country, Name AS GenreName, total_sold
FROM pop_genre_country pgc 
WHERE rnk = 1
ORDER BY Country ASC, total_sold DESC;

(25 rows affected)

Total execution time: 00:00:00.033

Country,GenreName,total_sold
Argentina,Rock,9
Argentina,Alternative & Punk,9
Australia,Rock,22
Austria,Rock,15
Belgium,Rock,21
Brazil,Rock,81
Canada,Rock,107
Chile,Rock,9
Czech Republic,Rock,25
Denmark,Rock,21


From the table above, it can be seen that the most popular genre in almost all the countries is Rock. How about we organise a Global Rock Concert for the customer in USA (where total sales of the genre is highest)? I bet we can gather a lot of people in it.

## Rock Fest Artists

Now that we have decided on organising a Rock Concert in USA, we can decide which musicians to invite to play at the concert. Let's invite the top 10 artists who have written the most rock music in our music store.

In [9]:
SELECT TOP(10) a.Name AS Artist, COUNT(*) AS number_of_songs
FROM Artist a 
JOIN Album al 
ON a.ArtistId = al.ArtistId
JOIN Track t 
ON al.AlbumId = t.AlbumId 
JOIN Genre g 
ON t.GenreId = g.GenreId
WHERE g.Name = 'Rock'
GROUP BY a.Name 
ORDER BY number_of_songs DESC;

(10 rows affected)

Total execution time: 00:00:00.011

Artist,number_of_songs
Led Zeppelin,114
U2,112
Deep Purple,92
Iron Maiden,81
Pearl Jam,54
Van Halen,52
Queen,45
The Rolling Stones,41
Creedence Clearwater Revival,40
Kiss,35


We can see the artists we should call at the concert. Better be sending those invites away soon.

## Customer Reward System

The management of Chinook wishes to reward the customers from each country who has spent the most on our platform with credit vouchers to encourage continued support and brand loyalty. Let's find who these customers are.

In [10]:
WITH customer_country AS 
(
    SELECT 
        c.Country, 
        c.CustomerId, 
        c.FirstName, 
        c.LastName,
        SUM(i.Total) AS total_spent,
        RANK() OVER(PARTITION BY c.Country ORDER BY SUM(i.Total) DESC) AS rnk

    FROM Customer c 
    JOIN Invoice i 
    ON c.CustomerId = i.CustomerId
    GROUP BY c.Country, c.CustomerId, c.FirstName, c.LastName
)

SELECT Country, CustomerId, (FirstName + ' ' + LastName) AS CustomerName, total_spent
FROM customer_country cc 
WHERE rnk = 1
ORDER BY Country, total_spent DESC;

(26 rows affected)

Total execution time: 00:00:00.011

Country,CustomerId,CustomerName,total_spent
Argentina,56,Diego Gutiérrez,37.62
Australia,55,Mark Taylor,37.62
Austria,7,Astrid Gruber,42.62
Belgium,8,Daan Peeters,37.62
Brazil,1,Luís Gonçalves,39.62
Canada,3,François Tremblay,39.62
Chile,57,Luis Rojas,46.62
Czech Republic,6,Helena Holý,49.62
Denmark,9,Kara Nielsen,37.62
Finland,44,Terhi Hämäläinen,41.62


So these were the top spenders in our platform from each country. Let's send 26 gift vuchers to these people as gratitude.